In [ ]:
file = 24 #Change number for file to run

In [ ]:
import numpy as np
import load_data as ld
import random
import matplotlib.pyplot as plt
from scipy import interpolate
from matplotlib.pyplot import figure

In [ ]:
def get_odometer_path(file):
    FR, FL, RR, RL, encoder_time  = ld.get_encoder('data/Encoders'+ str(file))

    width = 733
    data_length = len(encoder_time)
    
    Tic_Dist_Move = (np.pi*254)/360
    right_wheels = np.add(FR,RR)
    left_wheels = np.add(FL,RL)
    right_wheels_dist = (right_wheels*Tic_Dist_Move)/2
    left_wheels_dist  = (left_wheels*Tic_Dist_Move)/2
    
    Odo_Theta = np.subtract(right_wheels_dist,left_wheels_dist) / width
    
    for i in range(1,len(Odo_Theta)):
        Odo_Theta[i]+=Odo_Theta[i-1]
    
    Move_Forward = np.add(right_wheels_dist,left_wheels_dist) / 2.0
    Move_Direc_X = Move_Forward * np.cos(Odo_Theta)
    Move_Direc_Y = Move_Forward * np.sin(Odo_Theta) 

    for i in range(1,len(Move_Direc_X)):
        Move_Direc_X[i]+=Move_Direc_X[i-1]
        Move_Direc_Y[i]+=Move_Direc_Y[i-1]
    
    X_Odo_Path = Move_Direc_X * 0.001
    Y_Odo_Path = Move_Direc_Y * 0.001
       
    return X_Odo_Path, Y_Odo_Path, Odo_Theta, encoder_time

In [ ]:
def match_time(Lidar, Odo_Time):
    Lidar_Time = []
    Lidar_Index = []
    
    for i in range (len(Lidar)):
        Lidar_Time.append(Lidar[i]['t'])
    
    for i in range((Odo_Time).shape[0]):
        match = np.asarray(Lidar_Time)[(np.abs(Odo_Time[i] - np.asarray(Lidar_Time))).argmin()]
        Lidar_Index.append(int(Lidar_Time.index(match)))
        
    Time_Stamps = np.asarray(Lidar_Index)
    
    return Time_Stamps

In [ ]:
def polar2cart(scan, angle, Odo_Theta):
    X=(scan * np.cos(angle))*np.cos(Odo_Theta)-(scan * np.sin(angle))*np.sin(Odo_Theta)
    Y=(scan * np.cos(angle))*np.sin(Odo_Theta)+(scan * np.sin(angle))*np.cos(Odo_Theta)
    return X, Y

In [ ]:
def bresenham2D(sx, sy, ex, ey):
    sx = np.int(round(sx))
    sy = np.int(round(sy))
    ex = np.int(round(ex))
    ey = np.int(round(ey))
    dx = abs(ex-sx)
    dy = abs(ey-sy)
    steep = abs(dy)>abs(dx)
    if steep:
        dx,dy = dy,dx # swap 
    if dy == 0:
        q = np.zeros((dx+1,1))
    else:
        q = np.append(0,np.greater_equal(np.diff(np.mod(np.arange( np.floor(dx/2), -dy*dx+np.floor(dx/2)-1,-dy),dx)),0))
    if steep:
        if sy <= ey:
            y = np.arange(sy,ey+1)
        else:
            y = np.arange(sy,ey-1,-1)
        if sx <= ex:
            x = sx + np.cumsum(q)
        else:
            x = sx - np.cumsum(q)
    else:
        if sx <= ex:
            x = np.arange(sx,ex+1)
        else:
            x = np.arange(sx,ex-1,-1)
        if sy <= ey:
            y = sy + np.cumsum(q)
        else:
            y = sy - np.cumsum(q)
    return np.vstack((x,y)).astype(np.int)

In [ ]:
def create_map(Conv_Lidar ,X_Odo_Path,Y_Odo_Path,Size,MAP):
     
    for i in range(len(X_Odo_Path)):
        print(i)
               
        ScaleX = np.round(Conv_Lidar[i]['X']+X_Odo_Path[i],2)
        ScaleY = np.round(Conv_Lidar[i]['Y']+Y_Odo_Path[i],2)
        X_Pos = (ScaleX*10+Size/2).astype(np.int)
        Y_Pos = (ScaleY*10+Size/2).astype(np.int)
        
        for j in range(len(X_Pos)):       
            Stack = bresenham2D((X_Odo_Path[i]*10+Size/2),(Y_Odo_Path[i]*10+Size/2),X_Pos[j][0],Y_Pos[j][0])
            Stack_Miss = Stack[:, 0:(Stack.shape[1]-2)]
            Stack_Hit = Stack[:, (Stack.shape[1]-1)]
            MissY = Stack[0,:-1]
            MissX = Stack[1,:-1]
            HitY = Stack[0,-1:]
            HitX = Stack[1,-1:]

            if((Stack_Miss.size > 0) and (Stack_Hit.size > 0)):
                MAP[MissY,MissX] -= 0.1
                MAP[HitY,HitX] += 0.5
                
    return MAP

In [ ]:
X_Odo_Path, Y_Odo_Path, Odo_Theta, Odo_Time = get_odometer_path(file)

figure(figsize=(10, 10))
plt.plot(X_Odo_Path,Y_Odo_Path)
plt.xlabel('X-direction (m)')
plt.ylabel('Y-direction (m)')
plt.title('Path for file '+str(file))
plt.show()


In [ ]:
Lidar = ld.get_lidar('data/Hokuyo'+ str(file))
Time_Stamps  = match_time(Lidar, Odo_Time)    
Conv_Lidar = np.take(Lidar,Time_Stamps)

for i in range((len(Time_Stamps))):
    Lidar_Scans = np.reshape(Conv_Lidar[i]['scan'],(len(Conv_Lidar[i]['scan']),1))
    Lidar_Angles = np.reshape(Conv_Lidar[i]['angle'],(len(Conv_Lidar[i]['angle']),1))
    
    Conv_Lidar[i]['X'], Conv_Lidar[i]['Y'] = polar2cart(Lidar_Scans,Lidar_Angles,Odo_Theta[i])
    

Size = 850
MAP = np.zeros((Size,Size))
MAP = create_map(Conv_Lidar ,X_Odo_Path,Y_Odo_Path,Size, MAP)

MAP = np.where(MAP < 0, 0, MAP)
MAP = np.where(MAP > 20, 20, MAP)

convert_gray=interpolate.interp1d([-20,20],[0,255])
gray_map = convert_gray(MAP.T)

figure(figsize=(10, 10))
plt.title('Occupancy Grid and robot path for '+ str(file))
plt.xlabel('X-direction (m x 10)')
plt.ylabel('Y-direction (m x 10)')
plt.scatter(np.round(X_Odo_Path,2)*10+Size/2,np.round(Y_Odo_Path,2)*10+Size/2,s=0.1)
plt.imshow(gray_map,cmap='gray')
plt.show()

In [ ]:
figure(figsize=(10, 10))
plt.title('Occupancy Grid and robot path for '+ str(file))
plt.xlabel('X-direction (m x 10)')
plt.ylabel('Y-direction (m x 10)')
plt.scatter(np.round(X_Odo_Path,2)*10+Size/2,np.round(Y_Odo_Path,2)*10+Size/2,s=0.1)
plt.imshow(gray_map,cmap='gray')
plt.show()